<a href="https://colab.research.google.com/github/ankeyta03/Bank_loan_Case_Study/blob/main/gemaa_fb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install -q -U accelerate peft transformers bitsandbytes trl datasets torch torchvision torchaudio sentencepiece

# Check for GPU
import torch
print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: CUDA not available. Training will be extremely slow on CPU.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict

# Login to Hugging Face Hub
notebook_login()

# Define the direct URL to your JSON file on Hugging Face Hub
# This is the most robust way to load a specific file from a repo.
hf_dataset_file_url = "https://huggingface.co/datasets/Ankshi/mydataset/resolve/main/hr_interview_questions_dataset_nnew.json"

# Load dataset using the direct URL
try:
    # Use the "json" builder and provide the direct URL as data_files
    raw_datasets = load_dataset("json", data_files=hf_dataset_file_url, split="train")
    print(f"Dataset loaded. Train size: {len(raw_datasets) if 'train' not in raw_datasets else len(raw_datasets['train'])}")
except Exception as e:
    print(f"ERROR loading dataset: {e}")
    raise

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


hr_interview_questions_dataset_nnew.json:   0%|          | 0.00/247M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded. Train size: 500000


In [ ]:
import os
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
import torch

# raw_datasets is already a DatasetDict from Step 2.
# We no longer need to call .train_test_split() on it.
# We will ensure it has 'train' and 'test' splits for consistency.
if 'train' not in raw_datasets:
    raise ValueError("ERROR: 'train' split not found in raw_datasets. Ensure your dataset has a 'train' split.")

# If 'test' split is missing, create a small one from 'train'
if 'test' not in raw_datasets:
    print("WARNING: 'test' split not found. Creating a small test split from 'train' data.")
    temp_split = raw_datasets['train'].train_test_split(test_size=0.05, seed=42)
    raw_datasets = DatasetDict({
        'train': temp_split['train'],
        'test': temp_split['test']
    })

print("Dataset is ready for processing.")
print(f"Initial Train size: {len(raw_datasets['train'])}")
print(f"Initial Test size: {len(raw_datasets['test'])}")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("Tokenizer loaded.")

# Define the formatting and tokenization function
def format_and_tokenize_data(example):
    instruction_text = str(example["question"]).strip()
    response_text = str(example["ideal_answer"]).strip()

    if not instruction_text or not response_text:
        return {"input_ids": None}

    formatted_text = (
        f"<bos><start_of_turn>user\n{instruction_text}<end_of_turn>"
        f"<start_of_turn>model\n{response_text}<end_of_turn><eos>"
    )

    tokenized_output = tokenizer(
        formatted_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
        return_attention_mask=True
    )

    input_ids = tokenized_output["input_ids"].squeeze(0)
    labels = input_ids.clone()

    if input_ids.numel() == 0:
        return {"input_ids": None}

    return {
        "input_ids": input_ids,
        "attention_mask": tokenized_output["attention_mask"].squeeze(0),
        "labels": labels
    }

# Apply formatting and tokenization
print("Formatting and tokenizing dataset...")
formatted_and_tokenized_datasets = raw_datasets.map(
    format_and_tokenize_data,
    remove_columns=raw_datasets["train"].column_names, # Use train split's column names
    num_proc=4,
)

# Filter out problematic examples
formatted_and_tokenized_datasets = formatted_and_tokenized_datasets.filter(lambda x: x["input_ids"] is not None)

# Save processed datasets to disk
output_data_dir = "./processed_datasets"
os.makedirs(output_data_dir, exist_ok=True)
formatted_and_tokenized_datasets['train'].save_to_disk(os.path.join(output_data_dir, "train"))
formatted_and_tokenized_datasets['test'].save_to_disk(os.path.join(output_data_dir, "test"))
print(f"Processed datasets saved to {output_data_dir}")

# Load back from disk (optional, but good for stability)
formatted_datasets = DatasetDict({
    'train': Dataset.load_from_disk(os.path.join(output_data_dir, "train")),
    'test': Dataset.load_from_disk(os.path.join(output_data_dir, "test"))
})

print("Dataset processing complete.")
print(f"Train size: {len(formatted_datasets['train'])}")
print(f"Test size: {len(formatted_datasets['test'])}")
if len(formatted_datasets['train']) > 0:
    first_example_labels = formatted_datasets['train'][0]['labels']
    if isinstance(first_example_labels, list):
        first_example_labels = torch.tensor(first_example_labels)
    print(f"Example processed input_ids (first 10): {first_example_labels[:10].tolist()}")

Dataset is ready for processing.
Initial Train size: 475000
Initial Test size: 25000
Tokenizer loaded.
Formatting and tokenizing dataset...


Map (num_proc=4):   0%|          | 0/475000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/475000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/475000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Processed datasets saved to ./processed_datasets
Dataset processing complete.
Train size: 475000
Test size: 25000
Example processed input_ids (first 10): [2, 2, 106, 1645, 108, 2299, 749, 692, 17006, 577]


In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict

# Login to Hugging Face Hub (re-run this part)
notebook_login() # Paste your token here when prompted.

# Customize this with YOUR dataset ID (from your previous Step 2)
hf_dataset_file_url = "https://huggingface.co/datasets/Ankshi/mydataset/resolve/main/hr_interview_questions_dataset_nnew.json"

# Reload dataset (optional, but ensures context is fresh)
try:
    raw_datasets = load_dataset("json", data_files=hf_dataset_file_url, split="train")
    print("Dataset reloaded successfully.")
except Exception as e:
    print(f"Error reloading dataset: {e}")
    # Don't raise here, as the primary goal is re-login for model load.

Dataset reloaded successfully.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Model ID
model_id = "google/gemma-2b-it"

# LoRA Configuration
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "o_proj",
    "k_proj",
    "v_proj",
    "gate_proj",
    "up_proj",
    "down_proj"
]

# 4-bit Quantization Configuration (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Attach LoRA adapters
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=lora_target_modules,
)
model = get_peft_model(model, peft_config)

# Print trainable parameters summary
model.print_trainable_parameters()

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 9,805,824 || all params: 2,515,978,240 || trainable%: 0.3897


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset, DatasetDict
import os # Ensure os is imported if not already

# Load processed datasets from disk (saved in Step 3)
output_data_dir = "./processed_datasets"
formatted_datasets = DatasetDict({
    'train': Dataset.load_from_disk(os.path.join(output_data_dir, "train")),
    'test': Dataset.load_from_disk(os.path.join(output_data_dir, "test"))
})
print("Processed datasets loaded for training.")

# Define Training Arguments - OPTIMIZED FOR EXTREME SPEED AND MINIMAL LOGGING
training_output_dir = "./gemma_hr_interview_fine_tuned_model"
training_arguments = TrainingArguments(
    output_dir=training_output_dir,
    num_train_epochs=1,
    max_steps=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=5,                         # <--- THIS CONTROLS HOW OFTEN THE LOSS IS LOGGED
    save_steps=-1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",                        # <--- THIS ENSURES NO EXTRA REPORTING (e.g., to TensorBoard)
    eval_strategy="no",                      # <--- THIS ENSURES NO EVALUATION METRICS ARE LOGGED
    load_best_model_at_end=False,
    push_to_hub=False,
)
print("Training arguments defined for a very fast test run.")

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_datasets["train"],
    eval_dataset=formatted_datasets["test"],
    peft_config=peft_config,
    args=training_arguments,
)
print("SFTTrainer initialized. Starting training...")

# Start Training
trainer.train()

# Save the Fine-Tuned Adapters
final_model_save_path = os.path.join(training_output_dir, "gemma-2b-it-hr-interview-fine-tuned-adapters-new")
trainer.model.save_pretrained(final_model_save_path)
tokenizer.save_pretrained(final_model_save_path)
print(f"Fine-tuned adapters saved to: {final_model_save_path}")

Processed datasets loaded for training.
Training arguments defined for a very fast test run.


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


SFTTrainer initialized. Starting training...


Step,Training Loss
5,0.607300
10,0.306600
15,0.191100
20,0.155300


Fine-tuned adapters saved to: ./gemma_hr_interview_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new


In [ ]:
from google.colab import drive
import os
import shutil

# --- 1. Mount Google Drive ---
print("--- Mounting Google Drive ---")
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure your Google Drive is accessible and you have granted the necessary permissions.")
    exit()  # Stop execution if mounting fails

# --- 2. Define Save Paths ---
# Path where you want to save your model on Google Drive
# It's recommended to create a specific folder in your Drive first (e.g., 'my_fine_tuned_models')
gdrive_save_dir = "/content/drive/MyDrive/gemma_hr_fine_tuned_model"
print(f"Google Drive save directory: {gdrive_save_dir}")

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(gdrive_save_dir):
    try:
        os.makedirs(gdrive_save_dir)
        print(f"Created directory: {gdrive_save_dir}")
    except OSError as e:
        print(f"Error creating directory: {e}")
        print("Please check if you have write permissions in your Google Drive.")
        exit()

# Path to your locally saved fine-tuned adapters
local_model_path = "./gemma_hr_interview_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new"

# --- 3. Copy the Model and Tokenizer to Google Drive ---
print(f"\n--- Copying model files from '{local_model_path}' to Google Drive ---")

# Use shutil.copytree for recursive copying, handling potential errors
try:
    shutil.copytree(local_model_path, os.path.join(gdrive_save_dir, os.path.basename(local_model_path)))
    print(f"Model and tokenizer saved to Google Drive at: {os.path.join(gdrive_save_dir, os.path.basename(local_model_path))}")
    print("\nYou can find these files in your Google Drive, under 'My Drive' -> 'gemma_hr_fine_tuned_model'.")

except OSError as e:
    print(f"Error copying files: {e}")
    print("Please ensure the source path exists and is a valid directory.")

--- Mounting Google Drive ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Google Drive save directory: /content/drive/MyDrive/gemma_hr_fine_tuned_model
Created directory: /content/drive/MyDrive/gemma_hr_fine_tuned_model

--- Copying model files from './gemma_hr_interview_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new' to Google Drive ---
Model and tokenizer saved to Google Drive at: /content/drive/MyDrive/gemma_hr_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new

You can find these files in your Google Drive, under 'My Drive' -> 'gemma_hr_fine_tuned_model'.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# --- 1. Define Model Paths and IDs ---
base_model_id = "google/gemma-2b-it"
# Make sure this path points to where your fine-tuned adapters are saved locally
# This should be the same path as 'final_model_save_path' from your Step 5 code
fine_tuned_adapters_path = "./gemma_hr_interview_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new"

# --- 2. Load Tokenizer ---
print("--- Loading Tokenizer ---")
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# --- 3. Load the Original Base Model ---
# Load the base model (e.g., in float16 for reasonable memory usage)
print("\n--- Loading Original Base Gemma Model (for comparison) ---")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto", # Loads the model onto your GPU if available
    trust_remote_code=True,
)
base_model.eval() # Set to evaluation mode
print("Original Base Model loaded.")

# --- 4. Load the Fine-Tuned Model (Base Model + Adapters) ---
print(f"\n--- Loading Fine-Tuned Model from Adapters: {fine_tuned_adapters_path} ---")
# Load the base model again to apply adapters, or use a new instance
fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
# Load the PEFT adapters onto the base model
fine_tuned_model = PeftModel.from_pretrained(fine_tuned_model, fine_tuned_adapters_path)
fine_tuned_model.eval() # Set to evaluation mode
print("Fine-Tuned Model loaded and adapters applied.")

# --- 5. Prepare an Example Prompt ---
# Use a question similar to your fine-tuning dataset
prompt = "What are your strengths and weaknesses?"
formatted_prompt = f"<bos><start_of_turn>user\n{prompt}<end_of_turn><start_of_turn>model\n"

# --- 6. Generate Responses ---

# Generate response from Base Model
print("\n--- Generating response from ORIGINAL Base Model ---")
base_inputs = tokenizer(formatted_prompt, return_tensors="pt").to(base_model.device)
base_outputs = base_model.generate(
    **base_inputs,
    max_new_tokens=150, # Generate up to 150 new tokens
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
base_response = tokenizer.decode(base_outputs[0], skip_special_tokens=True)
print("Original Base Model Response:")
print(base_response.replace("<bos><start_of_turn>user\n" + prompt + "<end_of_turn><start_of_turn>model\n", "").strip())


# Generate response from Fine-Tuned Model
print("\n--- Generating response from FINE-TUNED Model ---")
fine_tuned_inputs = tokenizer(formatted_prompt, return_tensors="pt").to(fine_tuned_model.device)
fine_tuned_outputs = fine_tuned_model.generate(
    **fine_tuned_inputs,
    max_new_tokens=150, # Generate up to 150 new tokens
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
fine_tuned_response = tokenizer.decode(fine_tuned_outputs[0], skip_special_tokens=True)
print("Fine-Tuned Model Response:")
print(fine_tuned_response.replace("<bos><start_of_turn>user\n" + prompt + "<end_of_turn><start_of_turn>model\n", "").strip())

# --- 7. Compare and Conclude ---
print("\n--- Comparison ---")
print("Observe the differences in style, content, and directness of the answers.")
print("A successfully fine-tuned model should show responses that align more with the 'ideal_answer' format and content from your training data.")

# Clean up memory
del base_model
del fine_tuned_model
del base_inputs
del fine_tuned_inputs
del base_outputs
del fine_tuned_outputs
torch.cuda.empty_cache()
print("\nMemory cleared.")

--- Loading Tokenizer ---

--- Loading Original Base Gemma Model (for comparison) ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Original Base Model loaded.

--- Loading Fine-Tuned Model from Adapters: ./gemma_hr_interview_fine_tuned_model/gemma-2b-it-hr-interview-fine-tuned-adapters-new ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fine-Tuned Model loaded and adapters applied.

--- Generating response from ORIGINAL Base Model ---
Original Base Model Response:
user
What are your strengths and weaknesses?model
I do not have the ability to experience life or possess human strengths or weaknesses in the same way that humans do. I am an artificial intelligence and do not have a physical body, emotions, or the capacity to learn and grow.

**Strengths:**

* **Data-driven:** I am trained on a massive dataset of text and code, allowing me to access and process vast amounts of information.
* **Accuracy and precision:** I strive for high accuracy in my responses, providing factual and reliable information.
* **Objectivity:** I am not influenced by biases or opinions, offering a neutral and unbiased perspective.
* **Efficiency:** I can process and generate text quickly and efficiently, making me a valuable tool for communication, research, and

--- Generating response from FINE-TUNED Model ---
Fine-Tuned Model Response:
user